<a href="https://colab.research.google.com/github/dimang777/KaggleTF2/blob/master/tensorflowQA_mvp_bert_joint_on_nq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install kaggle
!pip install kaggle-cli
#!pip install tensorflow-gpu
# !pip install tf-nightly-gpu
!pip install tensorflow==1.15

In [0]:
import json
import zipfile
import os
import tensorflow as tf

print(tf.__version__)

1.15.0


In [0]:
os.environ['KAGGLE_USERNAME'] = "dimang777"
os.environ['KAGGLE_KEY'] = "a125f5747bf0877af81fc681e909777a"

!kaggle competitions download -c tensorflow2-question-answering # get the data

  0% 0.00/18.2k [00:00<?, ?B/s]
100% 18.2k/18.2k [00:00<00:00, 32.2MB/s]
100% 4.78M/4.78M [00:00<00:00, 30.7MB/s]

100% 4.46G/4.46G [01:35<00:00, 52.2MB/s]
100% 4.46G/4.46G [01:35<00:00, 50.4MB/s]


In [0]:
!ls /content
!ls / ## Isaac

sample_data	       simplified-nq-test.jsonl.zip
sample_submission.csv  simplified-nq-train.jsonl.zip
bin					   etc	  opt	 sys
boot					   home   proc	 tensorflow-2.1.0
content					   lib	  root	 tmp
datalab					   lib32  run	 tools
dev					   lib64  sbin	 usr
dlib-19.18.0-cp27-cp27mu-linux_x86_64.whl  media  srv	 var
dlib-19.18.0-cp36-cp36m-linux_x86_64.whl   mnt	  swift


In [0]:
!zcat simplified-nq-test.jsonl.zip > /content/simplified-nq-test.jsonl
!zcat simplified-nq-train.jsonl.zip | head -n 10000 > /content/simplified-nq-train_short.jsonl # unzip and make a sample of 10000 records for testing


gzip: simplified-nq-test.jsonl.zip: invalid compressed data--length error


In [0]:
!head /content/simplified-nq-train_short.jsonl
!head /content/simplified-nq-test.jsonl ## Isaac

{"document_text": "Email marketing - Wikipedia <H1> Email marketing </H1> Jump to : navigation , search <Table> <Tr> <Td> </Td> <Td> ( hide ) This article has multiple issues . Please help improve it or discuss these issues on the talk page . ( Learn how and when to remove these template messages ) <Table> <Tr> <Td> </Td> <Td> This article needs additional citations for verification . Please help improve this article by adding citations to reliable sources . Unsourced material may be challenged and removed . ( September 2014 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> <Table> <Tr> <Td> </Td> <Td> This article possibly contains original research . Please improve it by verifying the claims made and adding inline citations . Statements consisting only of original research should be removed . ( January 2015 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> ( Learn how and when to remove this template message ) </Td> </Tr> </Tabl

In [0]:
!wc -l /content/simplified-nq-train_short.jsonl # validate the number of records -- 10000 of 307373 records

10000 /content/simplified-nq-train_short.jsonl


In [0]:
import pandas as pd


In [0]:
data_line = []
nrows = 10000

# read the data
with open('/content/simplified-nq-train_short.jsonl', 'rt') as f:
    for i in range(nrows):
        data_line.append(json.loads(f.readline()))
 
train_df = pd.DataFrame(data_line) # convert to data frame

In [0]:
train_df # print train df

,document_text,long_answer_candidates,question_text,annotations,document_url,example_id
0,Email marketing - Wikipedia <H1> Email marketi...,"[{'start_token': 14, 'top_level': True, 'end_t...",which is the most common use of opt-in e-mail ...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Em...,5655493461695504401
1,The Mother ( How I Met Your Mother ) - wikiped...,"[{'start_token': 28, 'top_level': True, 'end_t...",how i.met your mother who is the mother,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Th...,5328212470870865242
2,Human fertilization - wikipedia <H1> Human fer...,"[{'start_token': 14, 'top_level': True, 'end_t...",what type of fertilisation takes place in humans,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Hu...,4435104480114867852
3,List of National Football League career quarte...,"[{'start_token': 28, 'top_level': True, 'end_t...",who had the most wins in the nfl,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Li...,5289242154789678439
4,Roanoke Colony - wikipedia <H1> Roanoke Colony...,"[{'start_token': 32, 'top_level': True, 'end_t...",what happened to the lost settlement of roanoke,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Ro...,5489863933082811018
...,...,...,...,...,...,...
9995,Numerical digit - wikipedia <H1> Numerical dig...,"[{'start_token': 14, 'top_level': True, 'end_t...",where did our digits for numbers come from,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Nu...,1748673986191848736
9996,Sakoku edict of 1635 - wikipedia <H1> Sakoku e...,"[{'start_token': 18, 'top_level': True, 'end_t...",when was the edict of 1635 ordering the closin...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Sa...,2651578436801931545
9997,Zodiac ( film ) - Wikipedia <H1> Zodiac ( film...,"[{'start_token': 18, 'top_level': True, 'end_t...",who plays the killer in the zodiac movie,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Zo...,-4175483072184014310
9998,Constitutional amendment - wikipedia <H1> Cons...,"[{'start_token': 8, 'top_level': True, 'end_to...",what is amendment why are amendments made in t...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Co...,9065261882686394678


In [0]:
!pip install --no-deps bert-tensorflow natural-questions

     |████████████████████████████████| 71kB 3.5MB/s 


In [0]:
## Isaac - entire cell
!gsutil ls gs://bert-nq/bert-joint-baseline
!gsutil ls gs://bert-nq/
!gsutil ls gs://bert-nq/tiny-dev

# !kaggle kernels pull philculliton/using-tensorflow-2-0-w-bert-on-nq


gs://bert-nq/bert-joint-baseline/bert_config.json
gs://bert-nq/bert-joint-baseline/bert_joint.ckpt.data-00000-of-00001
gs://bert-nq/bert-joint-baseline/bert_joint.ckpt.index
gs://bert-nq/bert-joint-baseline/nq-train.tfrecords-00000-of-00001
gs://bert-nq/bert-joint-baseline/vocab-nq.txt
gs://bert-nq/bert-joint-baseline_$folder$
gs://bert-nq/bert-joint-baseline/
gs://bert-nq/tiny-dev/
gs://bert-nq/tiny-dev/nq-dev-sample.jsonl.gz
gs://bert-nq/tiny-dev/nq-dev-sample.no-annot.jsonl.gz


In [0]:
## Isaac - entire cell
!gsutil cp -R gs://bert-nq/tiny-dev .


Copying gs://bert-nq/tiny-dev/nq-dev-sample.jsonl.gz...
Copying gs://bert-nq/tiny-dev/nq-dev-sample.no-annot.jsonl.gz...
\ [2 files][ 50.2 MiB/ 50.2 MiB]                                                
Operation completed over 2 objects/50.2 MiB.                                     


In [0]:
!gsutil cp -R gs://bert-nq/bert-joint-baseline .

Copying gs://bert-nq/bert-joint-baseline/bert_config.json...
Copying gs://bert-nq/bert-joint-baseline/bert_joint.ckpt.data-00000-of-00001...
Copying gs://bert-nq/bert-joint-baseline/bert_joint.ckpt.index...
Copying gs://bert-nq/bert-joint-baseline/nq-train.tfrecords-00000-of-00001...
/ [4 files][  2.3 GiB/  2.3 GiB]  153.1 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://bert-nq/bert-joint-baseline/vocab-nq.txt...
- [5 files][  2.3 GiB/  2.3 GiB]    1.1 MiB/s                                   
Operation completed over 5 objects/2.3 GiB.                                      


In [0]:
!ls /content/bert-joint-baseline/
!ls /content/ ## Isaac

bert_config.json		     nq-train.tfrecords-00000-of-00001
bert_joint.ckpt.data-00000-of-00001  vocab-nq.txt
bert_joint.ckpt.index
bert-joint-baseline	  simplified-nq-test.jsonl.zip
sample_data		  simplified-nq-train.jsonl.zip
sample_submission.csv	  simplified-nq-train_short.jsonl
simplified-nq-test.jsonl  tiny-dev


In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# !ls /gdrive/'My Drive'/kaggle/BERT_TF2/
!ls /gdrive/'My Drive'/kaggle/BERT_ON_NQ/

modelling.py  optimization.py  run_nq.py  tokenization.py


In [0]:
!cp -r /gdrive/'My Drive'/kaggle/BERT_ON_NQ/. /content/
!ls /content/
!ls /content/bert-joint-baseline ## Isaac

bert-joint-baseline    simplified-nq-test.jsonl
modelling.py	       simplified-nq-test.jsonl.zip
optimization.py        simplified-nq-train.jsonl.zip
run_nq.py	       simplified-nq-train_short.jsonl
sample_data	       tiny-dev
sample_submission.csv  tokenization.py
bert_config.json		     nq-train.tfrecords-00000-of-00001
bert_joint.ckpt.data-00000-of-00001  vocab-nq.txt
bert_joint.ckpt.index


# Using Tensorflow 2.0 with Bert on Natural Questions
https://www.kaggle.com/philculliton/using-tensorflow-2-0-w-bert-on-nq

In [0]:
import tensorflow as tf
# import tf2_0_baseline_w_bert as tf2baseline # old script
import run_nq as tf2baseline # my script
import modelling as modeling
import optimization as optimization
import tokenization as tokenization
import json
import absl
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random 
import datetime

import numpy as np

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# In this case, we've got some extra BERT model files under `/kaggle/input/bertjointbaseline`

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(absl.flags.FLAGS)

flags = absl.flags

## Isaac
kaggleserver = 0
if kaggleserver:
    address = "/kaggle/input/bertjointbaseline/"
    address2 = "/kaggle/input/tensorflow2-question-answering/"
    address3 = "/kaggle/working/"
else:
    address = "/content/bert-joint-baseline/"
    address2 = "/content/"
    address3 = "/content/"
## Isaac - end

## Isaac
flags.DEFINE_string(
    "bert_config_file", address+"bert_config.json",
    "The config json file corresponding to the pre-trained BERT model. "
    "This specifies the model architecture.")

## Isaac
flags.DEFINE_string("vocab_file", address+"vocab-nq.txt",
                    "The vocabulary file that the BERT model was trained on.")

flags.DEFINE_string(
    "output_dir", "outdir",
    "The output directory where the model checkpoints will be written.")

################################################################################
################################## Train data ##################################
################################################################################
flags.DEFINE_string("train_precomputed_file", 
                    "/content/bert-joint-baseline/nq-train.tfrecords-00000-of-00001",
                    "Precomputed tf records for training.")

flags.DEFINE_integer("train_num_precomputed", 494670,
                     "Number of precomputed tf records for training.")

# CHANGED - originally 3
flags.DEFINE_float("num_train_epochs", 1.0,
                    "Total number of training epochs to perform.")

# CHANGED - taken from the BERT on nq - can be tuned - original 5e-5
flags.DEFINE_float("learning_rate", 3e-5, "The initial learning rate for Adam.")
################################################################################


flags.DEFINE_string(
    "output_prediction_file", "predictions.json",
    "Where to print predictions in NQ prediction format, to be passed to"
    "natural_questions.nq_eval.")

## Isaac
flags.DEFINE_string(
    "init_checkpoint", address+"bert_joint.ckpt",
    "Initial checkpoint (usually from a pre-trained BERT model).")

flags.DEFINE_bool(
    "do_lower_case", True,
    "Whether to lower case the input text. Should be True for uncased "
    "models and False for cased models.")


# CHANGED - originally 384
flags.DEFINE_integer(
    "max_seq_length", 512,
    "The maximum total input sequence length after WordPiece tokenization. "
    "Sequences longer than this will be truncated, and sequences shorter "
    "than this will be padded.")

flags.DEFINE_integer(
    "doc_stride", 128,
    "When splitting up a long document into chunks, how much stride to "
    "take between chunks.")

flags.DEFINE_integer(
    "max_query_length", 64,
    "The maximum number of tokens for the question. Questions longer than "
    "this will be truncated to this length.")

################################################################################
flags.DEFINE_bool("do_train", True, "Whether to run training.")

flags.DEFINE_bool("do_predict", False, "Whether to run eval on the dev set.")
################################################################################

flags.DEFINE_integer("train_batch_size", 32, "Total batch size for training.")

flags.DEFINE_integer("predict_batch_size", 8,
                     "Total batch size for predictions.")

flags.DEFINE_float(
    "warmup_proportion", 0.1,
    "Proportion of training to perform linear learning rate warmup for. "
    "E.g., 0.1 = 10% of training.")

# CHANGED - originally 1000
flags.DEFINE_integer("save_checkpoints_steps", 5000,
                      "How often to save the model checkpoint.")

flags.DEFINE_integer("iterations_per_loop", 1000,
                     "How many steps to make in each estimator call.")

flags.DEFINE_integer(
    "n_best_size", 20,
    "The total number of n-best predictions to generate in the "
    "nbest_predictions.json output file.")

flags.DEFINE_integer(
    "verbosity", 1, "How verbose our error messages should be")

flags.DEFINE_integer(
    "max_answer_length", 30,
    "The maximum length of an answer that can be generated. This is needed "
    "because the start and end predictions are not conditioned on one another.")

flags.DEFINE_float(
    "include_unknowns", -1.0,
    "If positive, probability of including answers of type `UNKNOWN`.")

flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")
flags.DEFINE_bool("use_one_hot_embeddings", False, "Whether to use use_one_hot_embeddings")

absl.flags.DEFINE_string(
    "gcp_project", None,
    "[Optional] Project name for the Cloud TPU-enabled project. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

flags.DEFINE_bool(
    "verbose_logging", False,
    "If true, all of the warnings related to data processing will be printed. "
    "A number of warnings are expected for a normal NQ evaluation.")

flags.DEFINE_boolean(
    "skip_nested_contexts", True,
    "Completely ignore context that are not top level nodes in the page.")

flags.DEFINE_integer("task_id", 0,
                     "Train and dev shard to read from and write to.")

flags.DEFINE_integer("max_contexts", 48,
                     "Maximum number of contexts to output for an example.")

flags.DEFINE_integer(
    "max_position", 50,
    "Maximum context position for which to generate special tokens.")

flags.DEFINE_string(
    "tpu_zone", None,
    "[Optional] GCE zone where the Cloud TPU is located in. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

flags.DEFINE_string(
    "tpu_name", None,
    "The Cloud TPU to use for training. This should be either the name "
    "used when creating the Cloud TPU, or a grpc://ip.address.of.tpu:8470 "
    "url.")

flags.DEFINE_integer(
    "num_tpu_cores", 8,
    "Only used if `use_tpu` is True. Total number of TPU cores to use.")

## Special flags - do not change

flags.DEFINE_string(
    "predict_file", address2+"simplified-nq-test.jsonl",
    "NQ json for predictions. E.g., dev-v1.1.jsonl.gz or test-v1.1.jsonl.gz")
flags.DEFINE_boolean("logtostderr", True, "Logs to stderr")
flags.DEFINE_boolean("undefok", True, "it's okay to be undefined")
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('HistoryManager.hist_file', '', 'kernel')

FLAGS = flags.FLAGS
FLAGS(sys.argv) # Parse the flags

['/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py']

Here, we:
Set up Bert
Read in the test set
Run it past the pre-built Bert model to create embeddings
Use those embeddings to make predictions
Write those predictions to predictions.json
Feel free to change the code below. Code for the tf2baseline.* functions is included in the tf2_0_baseline_w_bert utility script, and can be customized, whether by forking the utility script and updating it, or by creating your own non-tf2baseline versions in this kernel.

Note: the tf2_0_baseline_w_bert utility script contains code for training your own embeddings. Here that code is removed.

In [0]:
if 0:
  import collections
  def read_nq_examples(input_file, is_training):
    """Read a NQ json file into a list of NqExample."""
    input_paths = tf.io.gfile.glob(input_file)
    input_data = []

    def _open(path):
      if path.endswith(".gz"):
        return gzip.GzipFile(fileobj=tf.io.gfile.GFile(path, "rb"))
      else:
        return tf.io.gfile.GFile(path, "r")

    for path in input_paths:
      absl.logging.info("Reading: %s", path)
      with _open(path) as input_file:
        for index, line in enumerate(input_file):
          input_data.append(create_example_from_jsonl(line))
          print(input_data[index])
          if index > 0:
              break

    examples = []
    for entry in input_data:
      examples.extend(read_nq_entry(entry, is_training))
    print(examples[0])
    return examples





  class NqExample(object):
    """A single training/test example."""

    def __init__(self,
                example_id,
                qas_id,
                questions,
                doc_tokens,
                doc_tokens_map=None,
                answer=None,
                start_position=None,
                end_position=None):
      self.example_id = example_id
      self.qas_id = qas_id
      self.questions = questions
      self.doc_tokens = doc_tokens
      self.doc_tokens_map = doc_tokens_map
      self.answer = answer
      self.start_position = start_position
      self.end_position = end_position




  def get_candidate_type_and_position(e, idx):
    """Returns type and position info for the candidate at the given index."""
    if idx == -1:
      return "[NoLongAnswer]"
    else:
      return e["long_answer_candidates"][idx]["type_and_position"]





  def get_first_annotation(e):
    """Returns the first short or long answer in the example.

    Args:
      e: (dict) annotated example.

    Returns:
      annotation: (dict) selected annotation
      annotated_idx: (int) index of the first annotated candidate.
      annotated_sa: (tuple) char offset of the start and end token
          of the short answer. The end token is exclusive.
    """

    if "annotations" not in e:
        return None, -1, (-1, -1)

    positive_annotations = sorted(
        [a for a in e["annotations"] if has_long_answer(a)],
        key=lambda a: a["long_answer"]["candidate_index"])

    for a in positive_annotations:
      if a["short_answers"]:
        idx = a["long_answer"]["candidate_index"]
        start_token = a["short_answers"][0]["start_token"]
        end_token = a["short_answers"][-1]["end_token"]
        return a, idx, (token_to_char_offset(e, idx, start_token),
                        token_to_char_offset(e, idx, end_token) - 1)

    for a in positive_annotations:
      idx = a["long_answer"]["candidate_index"]
      return a, idx, (-1, -1)

    return None, -1, (-1, -1)





  def get_candidate_type(e, idx):
    """Returns the candidate's type: Table, Paragraph, List or Other."""
    c = e["long_answer_candidates"][idx]
    first_token = e["document_tokens"][c["start_token"]]["token"]
    if first_token == "<Table>":
      return "Table"
    elif first_token == "<P>":
      return "Paragraph"
    elif first_token in ("<Ul>", "<Dl>", "<Ol>"):
      return "List"
    elif first_token in ("<Tr>", "<Li>", "<Dd>", "<Dt>"):
      return "Other"
    else:
      absl.logging.warning("Unknoww candidate type found: %s", first_token)
      return "Other"





  TextSpan = collections.namedtuple("TextSpan", "token_positions text")





  def get_text_span(example, span):
    """Returns the text in the example's document in the given token span."""
    token_positions = []
    tokens = []
    for i in range(span["start_token"], span["end_token"]):
      t = example["document_tokens"][i]
      if not t["html_token"]:
        token_positions.append(i)
        token = t["token"].replace(" ", "")
        tokens.append(token)
    return TextSpan(token_positions, " ".join(tokens))





  def get_candidate_text(e, idx):
    """Returns a text representation of the candidate at the given index."""
    # No candidate at this index.
    if idx < 0 or idx >= len(e["long_answer_candidates"]):
      return TextSpan([], "")

    # This returns an actual candidate.
    return get_text_span(e, e["long_answer_candidates"][idx])





  def should_skip_context(e, idx):
    if (FLAGS.skip_nested_contexts and
        not e["long_answer_candidates"][idx]["top_level"]):
      return True
    elif not get_candidate_text(e, idx).text.strip():
      # Skip empty contexts.
      return True
    else:
      return False




  def candidates_iter(e):
    """Yield's the candidates that should not be skipped in an example."""
    for idx, c in enumerate(e["long_answer_candidates"]):
      if should_skip_context(e, idx):
        continue
      yield idx, c




  def create_example_from_jsonl(line):
    """Creates an NQ example from a given line of JSON."""
    e = json.loads(line, object_pairs_hook=collections.OrderedDict)
    document_tokens = e["document_text"].split(" ")
    e["document_tokens"] = []
    for token in document_tokens:
        e["document_tokens"].append({"token":token, "start_byte":-1, "end_byte":-1, "html_token":"<" in token})

    add_candidate_types_and_positions(e)
    annotation, annotated_idx, annotated_sa = get_first_annotation(e)

    # annotated_idx: index of the first annotated context, -1 if null.
    # annotated_sa: short answer start and end char offsets, (-1, -1) if null.
    question = {"input_text": e["question_text"]}
    answer = {
        "candidate_id": annotated_idx,
        "span_text": "",
        "span_start": -1,
        "span_end": -1,
        "input_text": "long",
    }

    # Yes/no answers are added in the input text.
    if annotation is not None:
      assert annotation["yes_no_answer"] in ("YES", "NO", "NONE")
      if annotation["yes_no_answer"] in ("YES", "NO"):
        answer["input_text"] = annotation["yes_no_answer"].lower()

    # Add a short answer if one was found.
    if annotated_sa != (-1, -1):
      answer["input_text"] = "short"
      span_text = get_candidate_text(e, annotated_idx).text
      answer["span_text"] = span_text[annotated_sa[0]:annotated_sa[1]]
      answer["span_start"] = annotated_sa[0]
      answer["span_end"] = annotated_sa[1]
      expected_answer_text = get_text_span(
          e, {
              "start_token": annotation["short_answers"][0]["start_token"],
              "end_token": annotation["short_answers"][-1]["end_token"],
          }).text
      assert expected_answer_text == answer["span_text"], (expected_answer_text,
                                                          answer["span_text"])

    # Add a long answer if one was found.
    elif annotation and annotation["long_answer"]["candidate_index"] >= 0:
      answer["span_text"] = get_candidate_text(e, annotated_idx).text
      answer["span_start"] = 0
      answer["span_end"] = len(answer["span_text"])

    context_idxs = [-1]
    context_list = [{"id": -1, "type": get_candidate_type_and_position(e, -1)}]
    context_list[-1]["text_map"], context_list[-1]["text"] = (
        get_candidate_text(e, -1))
    for idx, _ in candidates_iter(e):
      context = {"id": idx, "type": get_candidate_type_and_position(e, idx)}
      context["text_map"], context["text"] = get_candidate_text(e, idx)
      context_idxs.append(idx)
      context_list.append(context)
      if len(context_list) >= FLAGS.max_contexts:
        break

    if "document_title" not in e:
        e["document_title"] = e["example_id"]

    # Assemble example.
    example = {
        "name": e["document_title"],
        "id": str(e["example_id"]),
        "questions": [question],
        "answers": [answer],
        "has_correct_context": annotated_idx in context_idxs
    }

    single_map = []
    single_context = []
    offset = 0
    for context in context_list:
      single_map.extend([-1, -1])
      single_context.append("[ContextId=%d] %s" %
                            (context["id"], context["type"]))
      offset += len(single_context[-1]) + 1
      if context["id"] == annotated_idx:
        answer["span_start"] += offset
        answer["span_end"] += offset

      # Many contexts are empty once the HTML tags have been stripped, so we
      # want to skip those.
      if context["text"]:
        single_map.extend(context["text_map"])
        single_context.append(context["text"])
        offset += len(single_context[-1]) + 1

    example["contexts"] = " ".join(single_context)
    example["contexts_map"] = single_map
    if annotated_idx in context_idxs:
      expected = example["contexts"][answer["span_start"]:answer["span_end"]]

      # This is a sanity check to ensure that the calculated start and end
      # indices match the reported span text. If this assert fails, it is likely
      # a bug in the data preparation code above.
      assert expected == answer["span_text"], (expected, answer["span_text"])

    return example




  def read_nq_entry(entry, is_training):
    """Converts a NQ entry into a list of NqExamples."""

    def is_whitespace(c):
      return c in " \t\r\n" or ord(c) == 0x202F

    examples = []
    contexts_id = entry["id"]
    contexts = entry["contexts"]
    doc_tokens = []
    char_to_word_offset = []
    prev_is_whitespace = True
    for c in contexts:
      if is_whitespace(c):
        prev_is_whitespace = True
      else:
        if prev_is_whitespace:
          doc_tokens.append(c)
        else:
          doc_tokens[-1] += c
        prev_is_whitespace = False
      char_to_word_offset.append(len(doc_tokens) - 1)

    questions = []
    for i, question in enumerate(entry["questions"]):
      qas_id = "{}".format(contexts_id)
      question_text = question["input_text"]
      start_position = None
      end_position = None
      answer = None
      if is_training:
        answer_dict = entry["answers"][i]
        answer = make_nq_answer(contexts, answer_dict)

        # For now, only handle extractive, yes, and no.
        if answer is None or answer.offset is None:
          continue
        start_position = char_to_word_offset[answer.offset]
        end_position = char_to_word_offset[answer.offset + len(answer.text) - 1]

        # Only add answers where the text can be exactly recovered from the
        # document. If this CAN'T happen it's likely due to weird Unicode
        # stuff so we will just skip the example.
        #
        # Note that this means for training mode, every example is NOT
        # guaranteed to be preserved.
        actual_text = " ".join(doc_tokens[start_position:(end_position + 1)])
        cleaned_answer_text = " ".join(
            tokenization.whitespace_tokenize(answer.text))
        if actual_text.find(cleaned_answer_text) == -1:
          absl.logging.warning("Could not find answer: '%s' vs. '%s'", actual_text, cleaned_answer_text)
          continue

      questions.append(question_text)
      example = NqExample(
          example_id=int(contexts_id),
          qas_id=qas_id,
          questions=questions[:],
          doc_tokens=doc_tokens,
          doc_tokens_map=entry.get("contexts_map", None),
          answer=answer,
          start_position=start_position,
          end_position=end_position)
      examples.append(example)
    return examples




  def add_candidate_types_and_positions(e):
    """Adds type and position info to each candidate in the document."""
    counts = collections.defaultdict(int)
    for idx, c in candidates_iter(e):
      context_type = get_candidate_type(e, idx)
      if counts[context_type] < FLAGS.max_position:
        counts[context_type] += 1
      c["type_and_position"] = "[%s=%d]" % (context_type, counts[context_type])






In [0]:
## Isaac - entire cell - testing purposes
# eval_examples = tf2baseline.read_nq_examples(
#     input_file=FLAGS.predict_file, is_training=False)
# eval_examples = read_nq_examples(
#     input_file=FLAGS.predict_file, is_training=False)

## Isaac - entire cell - testing purposes
# print(eval_examples[0])
# eval_examples[0]

In [0]:
# bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)

# tf2baseline.validate_flags_or_throw(bert_config)
# tf.io.gfile.makedirs(FLAGS.output_dir)

# tokenizer = tokenization.FullTokenizer(
#     vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

# # TPU is still in version 1 mode. Skip this for now and comment.
# # Can convert to TF2.0 if needed.
# # tpu_cluster_resolver = None
# # if FLAGS.use_tpu and FLAGS.tpu_name:
# #   tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
# #       FLAGS.tpu_name, zone=FLAGS.tpu_zone, project=FLAGS.gcp_project)

# # is_per_host = tf.compat.v1.estimator.tpu.InputPipelineConfig.PER_HOST_V2

# run_config = tf.estimator.RunConfig(
#     model_dir=FLAGS.output_dir,
#     save_checkpoints_steps=FLAGS.save_checkpoints_steps)

# num_train_steps = None
# num_warmup_steps = None

# ################################################################################
# # Added for training ###########################################################
# if FLAGS.do_train:
#   num_train_features = FLAGS.train_num_precomputed
#   num_train_steps = int(num_train_features / FLAGS.train_batch_size *
#                         FLAGS.num_train_epochs)

# num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)
# ################################################################################

# model_fn = tf2baseline.model_fn_builder(
#     bert_config=bert_config,
#     init_checkpoint=FLAGS.init_checkpoint,
#     learning_rate=FLAGS.learning_rate,
#     num_train_steps=num_train_steps,
#     num_warmup_steps=num_warmup_steps,
#     use_tpu=FLAGS.use_tpu,
#     use_one_hot_embeddings=FLAGS.use_one_hot_embeddings)
# print('modelfn' + str(type(model_fn)))

# estimator = tf.estimator.Estimator(
#     model_fn=model_fn,
#     config=run_config,
#     params={'batch_size':FLAGS.train_batch_size})

# ################################################################################
# # Added for training ###########################################################
# import timeit
# if FLAGS.do_train:
#   start = timeit.timeit()
#   print("***** Running training on precomputed features *****")
#   print("  Num split examples = %d", num_train_features)
#   print("  Batch size = %d", FLAGS.train_batch_size)
#   print("  Num steps = %d", num_train_steps)
#   train_filenames = tf.gfile.Glob(FLAGS.train_precomputed_file)
#   train_input_fn = tf2baseline.input_fn_builder(
#       input_file=train_filenames,
#       seq_length=FLAGS.max_seq_length,
#       is_training=True,
#       drop_remainder=True)
#   print(type(train_input_fn))
#   # estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

#   end = timeit.timeit()
#   print(end - start)

# ################################################################################

# if FLAGS.do_predict:
#   if not FLAGS.output_prediction_file:
#     raise ValueError(
#         "--output_prediction_file must be defined in predict mode.")
    
#   eval_examples = tf2baseline.read_nq_examples(
#       input_file=FLAGS.predict_file, is_training=False)

#   print("FLAGS.predict_file", FLAGS.predict_file)

#   eval_writer = tf2baseline.FeatureWriter(
#       filename=os.path.join(FLAGS.output_dir, "eval.tf_record"),
#       is_training=False)
#   eval_features = []

#   def append_feature(feature):
#     eval_features.append(feature)
#     eval_writer.process_feature(feature)

#   num_spans_to_ids = tf2baseline.convert_examples_to_features(
#       examples=eval_examples,
#       tokenizer=tokenizer,
#       is_training=False,
#       output_fn=append_feature)
#   eval_writer.close()
#   eval_filename = eval_writer.filename

#   print("***** Running predictions *****")
#   print(f"  Num orig examples = %d" % len(eval_examples))
#   print(f"  Num split examples = %d" % len(eval_features))
#   print(f"  Batch size = %d" % FLAGS.predict_batch_size)
#   for spans, ids in num_spans_to_ids.items():
#     print(f"  Num split into %d = %d" % (spans, len(ids)))

#   predict_input_fn = tf2baseline.input_fn_builder(
#       input_file=eval_filename,
#       seq_length=FLAGS.max_seq_length,
#       is_training=False,
#       drop_remainder=False)

#   all_results = []

#   for result in estimator.predict(
#       predict_input_fn, yield_single_examples=True):
#     if len(all_results) % 1000 == 0:
#       print("Processing example: %d" % (len(all_results)))

#     unique_id = int(result["unique_ids"])
#     start_logits = [float(x) for x in result["start_logits"].flat]
#     end_logits = [float(x) for x in result["end_logits"].flat]
#     answer_type_logits = [float(x) for x in result["answer_type_logits"].flat]

#     all_results.append(
#         tf2baseline.RawResult(
#             unique_id=unique_id,
#             start_logits=start_logits,
#             end_logits=end_logits,
#             answer_type_logits=answer_type_logits))

#   print ("Going to candidates file")

#   candidates_dict = tf2baseline.read_candidates(FLAGS.predict_file)

#   print ("setting up eval features")


#   raw_dataset = tf.data.TFRecordDataset(eval_filename)
#   eval_features = []
#   for raw_record in raw_dataset:
#     eval_features.append(tf.train.Example.FromString(raw_record.numpy()))
    
#   print ("compute_pred_dict")

#   nq_pred_dict = tf2baseline.compute_pred_dict(candidates_dict, eval_features,
#                                    [r._asdict() for r in all_results])
#   predictions_json = {"predictions": list(nq_pred_dict.values())}

#   print ("writing json")

#   with tf.io.gfile.GFile(FLAGS.output_prediction_file, "w") as f:
#     json.dump(predictions_json, f, indent=4)

modelfn<class 'function'>
INFO:tensorflow:Using config: {'_model_dir': 'outdir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7c4bcd22b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
***** Running training on precomputed features *****
  Num split examples = %

In [0]:
from tensorflow.contrib import tpu as contrib_tpu

tf.flags.DEFINE_string("master", None, "[Optional] TensorFlow master URL.")

def validate_flags_or_throw(bert_config):
  """Validate the input FLAGS or throw an exception."""
  if not FLAGS.do_train and not FLAGS.do_predict:
    raise ValueError("At least one of `{do_train,do_predict}` must be True.")

  if FLAGS.do_train:
    if not FLAGS.train_precomputed_file:
      raise ValueError("If `do_train` is True, then `train_precomputed_file` "
                       "must be specified.")
    if not FLAGS.train_num_precomputed:
      raise ValueError("If `do_train` is True, then `train_num_precomputed` "
                       "must be specified.")

  if FLAGS.do_predict:
    if not FLAGS.predict_file:
      raise ValueError(
          "If `do_predict` is True, then `predict_file` must be specified.")

  if FLAGS.max_seq_length > bert_config.max_position_embeddings:
    raise ValueError(
        "Cannot use sequence length %d because the BERT model "
        "was only trained up to sequence length %d" %
        (FLAGS.max_seq_length, bert_config.max_position_embeddings))

  if FLAGS.max_seq_length <= FLAGS.max_query_length + 3:
    raise ValueError(
        "The max_seq_length (%d) must be greater than max_query_length "
        "(%d) + 3" % (FLAGS.max_seq_length, FLAGS.max_query_length))

In [0]:
  tf.logging.set_verbosity(tf.logging.INFO)
  bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)
  validate_flags_or_throw(bert_config)
  tf.gfile.MakeDirs(FLAGS.output_dir)

  tokenizer = tokenization.FullTokenizer(
      vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

  tpu_cluster_resolver = None
  if FLAGS.use_tpu and FLAGS.tpu_name:
    tpu_cluster_resolver = contrib_cluster_resolver.TPUClusterResolver(
        FLAGS.tpu_name, zone=FLAGS.tpu_zone, project=FLAGS.gcp_project)

  is_per_host = contrib_tpu.InputPipelineConfig.PER_HOST_V2
  run_config = contrib_tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      master=FLAGS.master,
      model_dir=FLAGS.output_dir,
      save_checkpoints_steps=FLAGS.save_checkpoints_steps,
      tpu_config=contrib_tpu.TPUConfig(
          iterations_per_loop=FLAGS.iterations_per_loop,
          num_shards=FLAGS.num_tpu_cores,
          per_host_input_for_training=is_per_host))

  num_train_steps = None
  num_warmup_steps = None
  if FLAGS.do_train:
    num_train_features = FLAGS.train_num_precomputed
    num_train_steps = int(num_train_features / FLAGS.train_batch_size *
                          FLAGS.num_train_epochs)

    num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)

  model_fn = tf2baseline.model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=FLAGS.init_checkpoint,
      learning_rate=FLAGS.learning_rate,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      use_tpu=FLAGS.use_tpu,
      use_one_hot_embeddings=FLAGS.use_tpu)

  # If TPU is not available, this falls back to normal Estimator on CPU or GPU.
  estimator = contrib_tpu.TPUEstimator(
      use_tpu=FLAGS.use_tpu,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=FLAGS.train_batch_size,
      predict_batch_size=FLAGS.predict_batch_size)

  if FLAGS.do_train:
    tf.logging.info("***** Running training on precomputed features *****")
    tf.logging.info("  Num split examples = %d", num_train_features)
    tf.logging.info("  Batch size = %d", FLAGS.train_batch_size)
    tf.logging.info("  Num steps = %d", num_train_steps)
    train_filenames = tf.gfile.Glob(FLAGS.train_precomputed_file)
    train_input_fn = tf2baseline.input_fn_builder(
        input_file=train_filenames,
        seq_length=FLAGS.max_seq_length,
        is_training=True,
        drop_remainder=True)
    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

INFO:tensorflow:Using config: {'_model_dir': 'outdir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7c531b8160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_

In [0]:
!ls outdir/ -l --block-size=M
!ls bert-joint-baseline/ -l --block-size=M

total 3886M
-rw-r--r-- 1 root root    1M Jan 16 19:58 checkpoint
-rw-r--r-- 1 root root   26M Jan 16 19:56 events.out.tfevents.1579204573.de82b5b34ae2
-rw-r--r-- 1 root root   18M Jan 16 19:56 graph.pbtxt
-rw-r--r-- 1 root root 3836M Jan 16 19:58 model.ckpt-0.data-00000-of-00001
-rw-r--r-- 1 root root    1M Jan 16 19:58 model.ckpt-0.index
-rw-r--r-- 1 root root    8M Jan 16 19:58 model.ckpt-0.meta
total 2345M
-rw-r--r-- 1 root root    1M Jan 16 19:35 bert_config.json
-rw-r--r-- 1 root root 1279M Jan 16 19:35 bert_joint.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root    1M Jan 16 19:35 bert_joint.ckpt.index
-rw-r--r-- 1 root root 1067M Jan 16 19:35 nq-train.tfrecords-00000-of-00001
-rw-r--r-- 1 root root    1M Jan 16 19:36 vocab-nq.txt


Now, we turn predictions.json into a submission.csv file.

In [0]:
test_answers_df = pd.read_json(address3+"predictions.json")

The Bert model produces a confidence score, which the Kaggle metric does not use. You, however, can use that score to determine which answers get submitted. See the limits commented out in create_short_answer and create_long_answer below for an example.

Values for confidence will range between 1.0 and 2.0.

In [0]:
def create_short_answer(entry):
    # if entry["short_answers_score"] < 1.5:
    #     return ""
    
    answer = []    
    for short_answer in entry["short_answers"]:
        if short_answer["start_token"] > -1:
            answer.append(str(short_answer["start_token"]) + ":" + str(short_answer["end_token"]))
    if entry["yes_no_answer"] != "NONE":
        answer.append(entry["yes_no_answer"])
    return " ".join(answer)

def create_long_answer(entry):
   # if entry["long_answer_score"] < 1.5:
   # return ""

    answer = []
    if entry["long_answer"]["start_token"] > -1:
        answer.append(str(entry["long_answer"]["start_token"]) + ":" + str(entry["long_answer"]["end_token"]))
    return " ".join(answer)
test_answers_df["long_answer_score"] = test_answers_df["predictions"].apply(lambda q: q["long_answer_score"])
test_answers_df["short_answer_score"] = test_answers_df["predictions"].apply(lambda q: q["short_answers_score"])
test_answers_df["long_answer_score"].describe()


count    346.000000
mean       0.901300
std        0.250180
min        0.000000
25%        0.866473
50%        0.946224
75%        1.027903
max        1.332606
Name: long_answer_score, dtype: float64

An example of what each sample's answers look like in prediction.json:

In [0]:
test_answers_df.predictions.values[0]

{'example_id': '-1220107454853145579',
 'long_answer': {'end_byte': -1,
  'end_token': 336,
  'start_byte': -1,
  'start_token': 240},
 'long_answer_score': 1.143175601959228,
 'short_answers': [{'end_byte': -1,
   'end_token': 275,
   'start_byte': -1,
   'start_token': 271}],
 'short_answers_score': 1.143175601959228,
 'yes_no_answer': 'NONE'}

We re-format the JSON answers to match the requirements for submission.

In [0]:
test_answers_df["long_answer"] = test_answers_df["predictions"].apply(create_long_answer)
test_answers_df["short_answer"] = test_answers_df["predictions"].apply(create_short_answer)
test_answers_df["example_id"] = test_answers_df["predictions"].apply(lambda q: str(q["example_id"]))

long_answers = dict(zip(test_answers_df["example_id"], test_answers_df["long_answer"]))
short_answers = dict(zip(test_answers_df["example_id"], test_answers_df["short_answer"]))

Then we add them to our sample submission. Recall that each sample has both a _long and _short entry in the sample submission, one for each type of answer.

In [0]:
sample_submission = pd.read_csv(address2+"sample_submission.csv")

long_prediction_strings = sample_submission[sample_submission["example_id"].str.contains("_long")].apply(lambda q: long_answers[q["example_id"].replace("_long", "")], axis=1)
short_prediction_strings = sample_submission[sample_submission["example_id"].str.contains("_short")].apply(lambda q: short_answers[q["example_id"].replace("_short", "")], axis=1)

sample_submission.loc[sample_submission["example_id"].str.contains("_long"), "PredictionString"] = long_prediction_strings
sample_submission.loc[sample_submission["example_id"].str.contains("_short"), "PredictionString"] = short_prediction_strings



And finally, we write out our submission!

In [0]:
sample_submission.to_csv("submission.csv", index=False)
sample_submission.head()
!cp /content/submission.csv /gdrive/'My Drive'/kaggle/

!cp -r /content/outdir/. /gdrive/'My Drive'/kaggle/outdir/

,example_id,PredictionString
0,-1011141123527297803_long,339:513
1,-1011141123527297803_short,427:440
2,-1028916936938579349_long,42:321
3,-1028916936938579349_short,244:268
4,-1055197305756217938_long,221:335


next job: get bert_utils(tf2_0_baseline_w_bert), modeling, tokenization, optimization...

In [0]:
!ls
!ls \outdir

eval.tf_record
